In [1]:
#Тест на регрессию
import pandas as pd
import numpy as np

df = pd.read_csv("botswana.tsv", delimiter='\t')

#Вопрос 1
df["religion"].count(), df["religion"].unique()
#Ответ - 4

(4361, array(['catholic', 'protestant', 'spirit', 'other'], dtype=object))

In [2]:
#Вопрос 2
dropna = df.dropna()
dropna.count()

ceb         1834
age         1834
educ        1834
religion    1834
idlnchld    1834
knowmeth    1834
usemeth     1834
evermarr    1834
agefm       1834
heduc       1834
urban       1834
electric    1834
radio       1834
tv          1834
bicycle     1834
dtype: int64

In [3]:
df["nevermarr"] = np.abs(df["evermarr"] - 1)
df.drop("evermarr", axis=1, inplace=True)
df["agefm"] = df["agefm"].fillna(0)

for index, row in df.iterrows():
    if row["nevermarr"] == 1 and np.isnan(row["heduc"]):
        df.set_value(index, "heduc", -1)
        
#Вопрос 3
len(df[np.isnan(df["heduc"])])

C:\bin\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


123

In [4]:
#Вопрос 4
df["idlnchld_noans"] = np.isnan(df["idlnchld"])*1
df["heduc_noans"] = np.isnan(df["heduc"])*1
df["usemeth_noans"] = np.isnan(df["usemeth"])*1

df["idlnchld"] = df["idlnchld"].fillna(-1)
df["heduc"] = df["heduc"].fillna(-2)
df["usemeth"] = df["usemeth"].fillna(-1)

df = df.dropna()
df.shape[0] * df.shape[1]

78264

In [24]:
df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [30]:
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

REGR_EXPR = 'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
m1 = smf.ols(REGR_EXPR, data=df).fit(cov_type="HC1")
print(m1.summary()) #Вопрос 5. 0.644 SUCCESS

#Вопрос 6. 3 фиктивных переменных о религии

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Thu, 04 Jun 2020   Prob (F-statistic):               0.00
Time:                        16:58:30   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [31]:
import statsmodels.stats.api as sms

#Вопрос 7. Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?
sms.het_breuschpagan(m1.resid, m1.model.exog)
# SUCCESS

(1120.120590939138,
 1.1452927633439797e-225,
 79.04622432815992,
 2.0791008841131832e-262)

In [33]:
#Вопрос 8 - удаляем радио, тв и ебанную религию
REGR_EXPR2 = 'ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans'
m2 = smf.ols(REGR_EXPR2, data=df).fit()

m1.compare_f_test(m2)

(0.919235778463167, 0.467230554727435, 5.0)

In [34]:
#Вопрос 9
REGR_EXPR3 = 'ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans + heduc_noans'

m3 = smf.ols(REGR_EXPR3, data=df).fit()

m2.compare_f_test(m3)

(92.89058230109713, 3.155200948040263e-40, 2.0)